<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#读取数据" data-toc-modified-id="读取数据-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>读取数据</a></span></li><li><span><a href="#反查表" data-toc-modified-id="反查表-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>反查表</a></span></li><li><span><a href="#链接到本页的页也链接了" data-toc-modified-id="链接到本页的页也链接了-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>链接到本页的页也链接了</a></span></li><li><span><a href="#这些页也链接了本页的链接的页" data-toc-modified-id="这些页也链接了本页的链接的页-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>这些页也链接了本页的链接的页</a></span></li></ul></div>

In [1]:
import numpy as np
import pickle

# 读取数据

读取已经解析好的维基百科数据。参见PageRank笔记。

In [2]:
import pickle
import numpy as np

with open("pass1.5.data", "rb") as f:
    page_titles, redirect_map = pickle.load(f)

page_title_indices = {}
for i, page_title in enumerate(page_titles):
    page_title_indices[page_title] = i
    
    
def get_index_from_title(title):
    while True:
        if title in page_title_indices:
            return page_title_indices[title]
        
        # don't use capitalize, it we lower the first char of a name
        ctitle = title[:1].upper() + title[1:]
        if ctitle != title:
            if ctitle in page_title_indices:
                return page_title_indices[ctitle]
            
        if title in redirect_map:
            title_ = redirect_map[title]
            # this is not a full dection of loop, but it's work
            if title_ == title or title_ == ctitle:
                break
            title = title_
        elif ctitle in redirect_map:
            title_ = redirect_map[ctitle]
            if title_ == title or title_ == ctitle:
                break
            title = title_
        else:
            break
                        
def get_title_from_index(index):
    return page_titles[index]

In [50]:
links = np.load("pass2.links.npy")
page_n_links = np.load("pass2.page_n_links.npy")

assert(len(page_n_links) == len(page_title_indices))
assert(page_n_links.sum() == len(links))

print("Articles:", len(page_n_links))
print("Links:", len(links))

page_start = np.zeros(len(page_n_links), dtype=np.int)
page_start[1:] = np.cumsum(page_n_links)[:-1]


def get_links_from_title(title):
    index = get_index_from_title(title)
    if index is not None:
        s = page_start[index]
        n = page_n_links[index]
        return [get_title_from_index(i) for i in links[s:s+n] ]

Articles: 6047510
Links: 170622101


In [51]:
print(get_links_from_title("PageRank"))

['Logarithmic scale', 'Algorithm', 'Google Search', 'Web page', 'Web search engine', 'Larry Page', 'Google Patents', 'Network theory', 'Weighting', 'Hyperlink', 'Set (abstract data type)', 'World Wide Web', 'Link building', 'Webgraph', 'CNN', 'USA.gov', 'Recursion', 'Backlink', 'HITS algorithm', 'Jon Kleinberg', 'Teoma', 'Ask.com', 'CLEVER project', 'TrustRank', 'Google Hummingbird', 'Eigenvalues and eigenvectors', 'Scientometrics', 'Thomas L. Saaty', 'Analytic hierarchy process', 'Cognitive model', 'Baidu', 'Robin Li', 'The New York Times', 'Forbes', 'Sergey Brin', 'Stanford University', 'Rajeev Motwani', 'Terry Winograd', 'Google', 'Software patent', 'Citation analysis', 'Eugene Garfield', 'Hyper Search', 'Massimo Marchiori', 'University of Padua', 'Probability distribution', 'Matt Cutts', 'Markov chain', 'URL', 'Adjacency matrix', 'Stochastic matrix', 'Eigenvector centrality', 'Eigengap', 'Expected value', 'Wikipedia', 'Link farm', 'Trade secret', 'Power iteration', 'Steady state', 

# 反查表

In [36]:
def make_inverse_table():
        
    n_pages = len(page_n_links)
    inverse_link = [[] for i in range(n_pages)]
    
    for lk in range(n_pages):        
        n_links = page_n_links[lk]
        start = page_start[lk]
        
        for j in range(start, start+n_links): 
            this = links[j]            
            inverse_link[this].append(lk)
    
    global links_inverse
    global page_n_links_inverse
    global page_start_inverse
    links_inverse = np.empty(shape=len(links), dtype=np.int)
    page_n_links_inverse = np.empty(shape=len(page_n_links), dtype=np.int)
    
    l = 0
    for i, lks in enumerate(inverse_link):
        page_n_links_inverse[i] = len(lks)
        if len(lks):
            links_inverse[l:l+len(lks)] = lks
            l += len(lks)
    
    page_start_inverse = np.zeros(len(page_n_links_inverse), dtype=np.int)
    page_start_inverse[1:] = np.cumsum(page_n_links_inverse)[:-1]
    
    assert page_n_links_inverse.sum() == len(links_inverse)
    
make_inverse_table()

In [53]:
def get_links_from_title_inverse(title):
    index = get_index_from_title(title)
    if index is not None:
        s = page_start_inverse[index]
        n = page_n_links_inverse[index]
        return [get_title_from_index(i) for i in links_inverse[s:s+n] ]

In [54]:
print(get_links_from_title_inverse("PageRank"))

['Carl Linnaeus', 'Google Search', 'History of the Internet', 'List of algorithms', 'Web crawler', 'Spamdexing', 'Hyperlink', 'South East England', 'Advogato', 'Markov chain', 'Larry Page', 'Sergey Brin', "Bradford's law", 'Search engine optimization', 'Google Answers', 'Spectrum of a matrix', 'Link farm', 'Google bombing', 'PR', 'MediaWiki', 'Google (verb)', '1996 in science', 'Newsgroup spam', 'Wassily Leontief', 'Timeline of algorithms', 'Relevance', 'Fixed point (mathematics)', 'Global Multimedia Protocols Group', 'Thomas L. Saaty', 'Spam in blogs', 'Backlink', 'MozDex', 'Google Toolbar', 'Automatic summarization', 'Webometrics', 'Affiliate marketing', 'Document retrieval', 'Network theory', 'Web mining', 'List of eponyms (L–Z)', 'Trust metric', 'List of University of California, Berkeley alumni', 'VoIP spam', 'Findability', 'Google', 'Bibliometrics', 'Citation analysis', 'Jon Kleinberg', 'Eugene Garfield', 'Full-text search', 'Referrer spam', 'Centrality', 'Ranking', 'List of Goog

# 链接到本页的页也链接了

In [87]:
def collect_links(i, links, page_n_links, page_start,
                  links_inverse, page_n_links_inverse, page_start_inverse,
                  links_):
    
    total = 0    
    start1 = page_start_inverse[i]
    n_pages1 = page_n_links_inverse[i]
    
    # all pages that links to this
    for l1 in range(start1, start1 + n_pages1):
        l1_ = links_inverse[l1]
        
        
        start2 = page_start[l1_]
        n_pages2 = page_n_links[l1_]
        
        # all pages that page(l1_) links to
        for l2 in range(start2, start2+n_pages2):
            l2_ = links[l2]
            links_[total] = l2_
            total += 1
    return total
    

def itemitem(i, links, page_n_links, page_start,
             links_inverse, page_n_links_inverse, page_start_inverse,
             links_ = None):
    
    if links_ is None:
        links_  = np.empty(len(links), np.int)
    
    total = collect_links(i, links, page_n_links, page_start,
                  links_inverse, page_n_links_inverse, page_start_inverse, links_)
    
    uns, uns_count = np.unique(links_[:total], return_counts=True)
    uns_w = uns_count/(np.sqrt(page_n_links_inverse[uns])*np.sqrt(page_n_links_inverse[i]))
    
    indices = np.argsort(uns_w)[::-1][:10]
    uns = uns[indices]
    return uns
    
def pages_link_to_this_page_also_link_to(this_page):
    i = get_index_from_title(this_page)
    uns = itemitem(i, links, page_n_links, page_start,
                  links_inverse, page_n_links_inverse, page_start_inverse)
    
    for pid in uns:
        page = get_title_from_index(pid)
        print(page)


In [107]:
pages_link_to_this_page_also_link_to(this_page="History of Beijing")
print("====")
pages_link_to_this_page_also_link_to(this_page="PageRank")
print("====")
pages_link_to_this_page_also_link_to(this_page="Liang (surname)")

History of Beijing
Beijing city fortifications
Ji (state)
Liu Bingzhong
Taiye Lake
Jingshan Park
Khanbaliq
Xicheng District
Imperial City, Beijing
Jicheng (Beijing)
====
PageRank
HITS algorithm
Spamdexing
Backlink
Google Panda
Google Penguin
TrustRank
Katz centrality
History of Google
Search engine optimization
====
Liang (surname)
Ge (surname)
Mou (surname)
Wat (surname)
Fei (surname)
Lian (surname)
Ji (surname 嵇)
Shuai
Liang Siyong
Di (surname)


# 这些页也链接了本页的链接的页

In [80]:
  def pages_this_page_link_to_also_linked_by(this_page):
    i = get_index_from_title(this_page)
    uns = itemitem(i, links_inverse, page_n_links_inverse, page_start_inverse,
                  links, page_n_links, page_start)
    
    for pid in uns:
        page = get_title_from_index(pid)
        print(page)


In [106]:
pages_this_page_link_to_also_linked_by(this_page="History of Beijing")
print("====")
pages_this_page_link_to_also_linked_by(this_page="PageRank")
print("====")
pages_this_page_link_to_also_linked_by(this_page="Liang (surname)")

History of Beijing
Jicheng (Beijing)
Beijing
Nanjing (Liao dynasty)
History of China
Beijing city fortifications
Timeline of Chinese history
History of the Great Wall of China
You Prefecture
Zhongnanhai
====
PageRank
TrustRank
Search engine optimization
Nofollow
Backlink
Hilltop algorithm
Google matrix
Spamdexing
Anchor text
HITS algorithm
====
Liang (surname)
Mediacorp Subaru Impreza WRX Challenge
Borders of Vietnam
Macrotermes carbonarius
Alstonia spatulata
Box of Hope
Gymnanthera oblonga
Culex rubithoracis
Miele Guide
Asian-Oceanian Computing Industry Organization
